# Using the cache

First make sure that the dependencies are installed:

```bash
pip install -r requirements-dev.txt
```

Then run redis using docker compose:

```bash
docker compose -f docker-compose.redis.yml up -d
```

Then run the notebook. 

If you want to view the data in redis, you can use the redis-cli:

```bash
brew install redis
redis-cli -p 6380
```

In [1]:
%%time

import fsspec
import json
import xarray as xr
from redis import Redis
from redis_fsspec_cache.sync import RedisCachingFileSystem
from redis_fsspec_cache.reference import RedisCachingReferenceFileSystem

redis = Redis(host="localhost", port=6380)

s3fs = fsspec.filesystem("s3", anon=True)
cached_s3fs = RedisCachingFileSystem(redis=redis, fs=s3fs, expiry_time=60)
raw_refs = cached_s3fs.cat_file('s3://nextgen-dmac-cloud-ingest/nos/ngofs2/nos.ngofs2.fields.best.nc.zarr')
refs = json.loads(raw_refs)

new_cached_fs = RedisCachingReferenceFileSystem(
    redis=redis,
    expiry_time=60,
    fo=refs, 
    remote_protocol='s3', 
    remote_options={'anon':True}, 
    target_protocol='s3', 
    target_options={'anon':True}, 
    asynchronous=True, 
)

m = new_cached_fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), 
                      chunks={}, drop_variables=['Itime2'])
ds

CPU times: user 1.2 s, sys: 2.97 s, total: 4.18 s
Wall time: 1.54 s


<xarray.Dataset> Size: 115GB
Dimensions:             (time: 261, four: 4, nele: 569405, node: 303714,
                         three: 3, maxnode: 10, maxelem: 8, siglev: 41,
                         siglay: 40)
Coordinates:
    lat                 (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    latc                (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    lon                 (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    lonc                (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    siglay              (siglay, node) float32 49MB dask.array<chunksize=(40, 303714), meta=np.ndarray>
    siglev              (siglev, node) float32 50MB dask.array<chunksize=(41, 303714), meta=np.ndarray>
  * time                (time) datetime64[ns] 2kB 2024-04-19T21:00:00 ... 202...
Dimensions without coordinates: four, nele, node, three, maxnode, maxelem
Data variables: (12/42)
    Itime               (time) datetime64[ns] 2kB dask.array<chunksize=(1,), meta=np.ndarray>
    Times               (time) |S26 7kB dask.array<chunksize=(1,), meta=np.ndarray>
    a1u                 (four, nele) float32 9MB dask.array<chunksize=(4, 569405), meta=np.ndarray>
    a2u                 (four, nele) float32 9MB dask.array<chunksize=(4, 569405), meta=np.ndarray>
    art1                (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    art2                (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    ...                  ...
    ww                  (time, siglay, nele) float32 24GB dask.array<chunksize=(1, 40, 569405), meta=np.ndarray>
    x                   (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    xc                  (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    y                   (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    yc                  (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    zeta                (time, node) float32 317MB dask.array<chunksize=(1, 303714), meta=np.ndarray>
Attributes: (12/14)
    Conventions:                 CF-1.0
    CoordinateProjection:        init=nad83:4205
    CoordinateSystem:            GeoReferenced
    GroundWater_Forcing:         GROUND WATER FORCING IS OFF!
    River_Forcing:               THERE ARE 63 RIVERS IN THIS MODEL.\nRIVER IN...
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    ...                          ...
    Tidal_Forcing:               TIDAL ELEVATION FORCING IS OFF!
    history:                     model started at: 19/04/2024   21:16
    institution:                 School for Marine Science and Technology
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    source:                      FVCOM_4.3
    title:                       NGOFS2

In [2]:
%%time
ds.lat.values

CPU times: user 111 ms, sys: 23.3 ms, total: 134 ms
Wall time: 826 ms


array([21.789427, 21.7978  , 21.81    , ..., 30.50772 , 30.50776 ,
       30.507713], dtype=float32)

In [3]:
%%time
ds.lat.values

CPU times: user 2.38 ms, sys: 2.51 ms, total: 4.89 ms
Wall time: 22.5 ms


array([21.789427, 21.7978  , 21.81    , ..., 30.50772 , 30.50776 ,
       30.507713], dtype=float32)

In [4]:
%%time
len(ds.nv.values)

CPU times: user 24.1 ms, sys: 18.2 ms, total: 42.3 ms
Wall time: 793 ms


3

In [5]:
%%time
len(ds.nv.values)

CPU times: user 4.57 ms, sys: 8.68 ms, total: 13.2 ms
Wall time: 40.2 ms


3

In [6]:
%%time

ds.zeta.isel(time=slice(0,10)).values

CPU times: user 90.6 ms, sys: 63 ms, total: 154 ms
Wall time: 1.53 s


array([[0.18136132, 0.18053554, 0.1788693 , ..., 4.0884604 , 4.077911  ,
        4.0913353 ],
       [0.07933186, 0.07882892, 0.07785864, ..., 4.253203  , 4.241439  ,
        4.256582  ],
       [0.07519985, 0.0749549 , 0.07437387, ..., 4.1548743 , 4.143965  ,
        4.15794   ],
       ...,
       [0.15917033, 0.15872553, 0.15802062, ..., 4.382236  , 4.369806  ,
        4.3861217 ],
       [0.1381165 , 0.13736528, 0.13643828, ..., 4.1747184 , 4.163902  ,
        4.177847  ],
       [0.04129069, 0.04099029, 0.04048311, ..., 4.335432  , 4.3234344 ,
        4.3391504 ]], dtype=float32)

In [7]:
%%time

ds.zeta.isel(time=slice(0,10)).values

CPU times: user 12.6 ms, sys: 14.5 ms, total: 27.1 ms
Wall time: 49.2 ms


array([[0.18136132, 0.18053554, 0.1788693 , ..., 4.0884604 , 4.077911  ,
        4.0913353 ],
       [0.07933186, 0.07882892, 0.07785864, ..., 4.253203  , 4.241439  ,
        4.256582  ],
       [0.07519985, 0.0749549 , 0.07437387, ..., 4.1548743 , 4.143965  ,
        4.15794   ],
       ...,
       [0.15917033, 0.15872553, 0.15802062, ..., 4.382236  , 4.369806  ,
        4.3861217 ],
       [0.1381165 , 0.13736528, 0.13643828, ..., 4.1747184 , 4.163902  ,
        4.177847  ],
       [0.04129069, 0.04099029, 0.04048311, ..., 4.335432  , 4.3234344 ,
        4.3391504 ]], dtype=float32)